## TensorFlow, PyTorch, PyMC3 Distributions

In [ ]:
import scipy.stats as ss
import scipy.optimize as so
from scipy.optimize import minimize, fmin
import numpy as np
import pandas as pd
import collections
import math
import matplotlib.pyplot as plt
import seaborn as sns

import pymc3 as pm

import torch
from torch.distributions import Bernoulli

import tensorflow as tf
import tensorflow_probability as tfp
tfd = tf.contrib.distributions
# tfd = tfp.distributions
# tfb = tfp.distributions.bijectors

tfe = tf.contrib.eager
tfe.enable_eager_execution()
# sess = tf.InteractiveSession()
# sess.run(tf.global_variables_initializer())

from __future__ import print_function

print(tf.__version__)

import warnings
warnings.filterwarnings(action='once')

##Helpful links for Distributions
# https://arxiv.org/pdf/1711.10604.pdf
# https://github.com/tensorflow/probability/blob/master/tensorflow_probability/examples/jupyter_notebooks/Understanding%20TensorFlow%20Distributions%20Shapes.ipynb
# https://research.googleblog.com/2017/10/eager-execution-imperative-define-by.html
# https://www.tensorflow.org/api_docs/python/tf/distributions/Normal

In [ ]:
##Tinker with tf distributions

dist = tf.distributions.Normal(loc=0., scale=3.)
print(dist.cdf(0.))
dist = tf.distributions.Normal(loc=[1, 2.], scale=[11, 22.])
# Evaluate the pdf of the first distribution on 0, and the second on 1.5, returning a length two tensor 
print(dist.prob([0, 1.5]))

# Get 3 samples, returning a 3 x 2 tensor.
print(dist.sample([3]))
print(dist.sample([3]).numpy())

In [ ]:
##Tinker with pytorch distributions

from torch.distributions import Bernoulli

m = Bernoulli(torch.Tensor([0.3]))
r = []
for i in range(20):
    r.append(m.sample().numpy()[0])
print(r)  # 30% chance 1; 70% chance 0

In [ ]:
##Tinker with tf mixture of distributions
# https://www.tensorflow.org/api_docs/python/tf/contrib/distributions/Mixture

mixture_components = [1.0/2]*2
bimix_gauss = tfd.Mixture(
  cat=tfd.Categorical(probs=mixture_components),
  components=[
    tfd.Normal(loc=-1., scale=0.1),
    tfd.Normal(loc=+1., scale=0.5)
])

#Is Weibull bijector the only way to directly use weibull distribution in tf ?

x = tf.linspace(-2., 3., int(1e4))
bimix_gauss.prob(x)

In [ ]:
##Tinker with PyMC distributions. Wrap scipy distribution in PyMC distribution

y = pm.Binomial.dist(n=10, p=0.5)
print(y.logp(5).eval(), y.logp(4).eval(), y.logp(6).eval())
print(y.logp_sum(5).eval(), y.logp_sum(4).eval(), y.logp_sum(6).eval())
print(y.random(size=10))

with pm.Model() as model:
    g = pm.Gamma('g', 1, 1)
print(model.vars, model.deterministics)

#Invoke scipy dist through PyMC 
print(ss.binom(10, 0.3).cdf(5))

import theano
with pm.Model() as model:
    g = pm.Gamma('g', 1, 1)
    b =  ss.binom(10, 0.3)
    z = b + g
    tmp_trace = pm.sample(1000)

pm.summary(tmp_trace)

## Scipy Distributions / Optimization

In [ ]:
print(np.random.binomial(10, 0.3, 10))
print(np.random.binomial(10, 0.3, 10).mean())

print(ss.binom(10, 0.3).rvs(10))
print(ss.binom(10, 0.3).cdf(5))
print(ss.binom(10, 0.3).mean())

In [ ]:
##Using scipy optimize with custom distributions
# https://www.projectrhea.org/rhea/index.php/MLE_Examples:_Exponential_and_Geometric_Distributions_Old_Kiwi

from scipy.stats import geom, gamma
from scipy.optimize import minimize

def neg_loglike(theta):
    prob = lambda x, y, z: -x * np.log(geom.pmf(y, z))
    data = [10, 9, 8, 7, 6, 5]
    return np.sum([prob(data[i], i+1, theta) for i in range(len(data))])

theta_start = 0.5
res = minimize(neg_loglike, theta_start, method = 'Nelder-Mead', options={'disp':True})
print(res)

print('------------')

def weibull_cdf(t, l, c):
    return 1 - math.exp(-l * (t**c))

def weibull_2seg_2clust_withspike_negloglike_dummy(theta): #TODO: Implement correct function later
    prob = lambda x, y, z: -x * np.log(gamma.pdf(y, z))
    data = [10, 9, 8, 7, 6, 5]
    return np.sum([prob(data[i], i+1, theta) for i in range(len(data))])

param_start = 0.5
res = minimize(weibull_2seg_2clust_withspike_negloglike_dummy, param_start, method = 'Nelder-Mead', options={'disp':True}) 
print(res)

In [ ]:
# https://github.com/scipy/scipy/issues/3056

import scipy.optimize as optimize

X = np.array([[1.020626, 1.013055], [0.989094, 1.059343]])
freq = 13.574380165289256
x_0 = [1., 1.]

def objective(b):
    def foo(r_log, freq):
        mu, sd = r_log.mean(), r_log.std()
        sd += 0.5 / freq
        return mu / sd * np.sqrt(freq)

    print(b)
    return -foo(np.log(np.maximum(np.dot(X - 1, b) + 1, 0.2)), freq=freq)

cons = ({'type': 'ineq', 'fun': lambda b: 2. - sum(b)},)
res = optimize.minimize(objective, x_0, bounds=[(0., 2.)]*len(x_0), constraints=cons, method='slsqp')
print(res)

## Generalized Linear Models

In [ ]:
##Poisson Regression
# https://stackoverflow.com/questions/47686227/poisson-regression-in-statsmodels-and-r

# import warnings
# warnings.filterwarnings('ignore')

import statsmodels.formula.api
import statsmodels.api as sm
from statsmodels.genmod.families import Poisson

df = pd.DataFrame(np.random.randint(100, size=(50,2)))
df.rename(columns={0:'X1', 1:'X2'}, inplace=True)

# glm = statsmodels.formula.api.gee
# model = glm("X2 ~ X1", groups=None, data=df, family=Poisson())
# results = model.fit()
# print(results.summary())

# Add a column of ones for the intercept to create input X. This model is similar to R implementation
X = np.column_stack( (np.ones((df.shape[0], 1)), df.X1) )
y = df.X2
print(sm.GLM(y, X, family = Poisson()).fit().summary())

## Paper Implementations

In [ ]:
##Try implementing following papers using scipy, octave, tensorflow, pytorch (whichever is flexbile, scalable)
#https://repository.upenn.edu/cgi/viewcontent.cgi?referer=https://www.google.com/&httpsredir=1&article=1048&context=wharton_research_scholars
#https://pdfs.semanticscholar.org/d566/7b80c85b221aedf9c498fdd1d98a4478118b.pdf

In [ ]:
#Dummy data
ts_data1 = np.array([8,7,6,5,4,3,3,3,3,4,5,6,7,8,9,10])
ts_data2 = np.concatenate((5+ts_data1[0:8], ts_data1[8:]))
ts_data3 = np.concatenate((ts_data1[0:8], 5+ts_data1[8:]))
ts_data = np.array([ts_data1.cumsum(), ts_data2.cumsum(), ts_data3.cumsum()])
ts_data = np.tile(ts_data,(5,1))
ts_data_sum = np.sum(ts_data, axis=1)

# ts_data_context = np.array(['a','b','b','b','b','c','c','c','c','c','d','d','d','d','d','e'])
print(len(ts_data), ts_data_sum) #,len(ts_data_context))

In [ ]:
num_clusters = 2
tot_num_clusters = 2 #num_clusters + 1
num_samples = len(ts_data)

clust_assign = np.random.randint(num_clusters, size=num_samples)

In [ ]:
X = np.array([[1.020626, 1.013055], [0.989094, 1.059343]])
freq = 13.574380165289256
x_0 = [1., 1.]

def objective(b):
    def foo(r_log, freq):
        mu, sd = r_log.mean(), r_log.std()
        sd += 0.5 / freq
        return mu / sd * np.sqrt(freq)

    print(b)
    return -foo(np.log(np.maximum(np.dot(X - 1, b) + 1, 0.2)), freq=freq)

cons = ({'type': 'ineq', 'fun': lambda b: 2. - sum(b)},)
res = so.minimize(objective, x_0, bounds=[(0., 2.)]*len(x_0), constraints=cons, method='slsqp')
print(res)

In [ ]:
# https://github.com/pymc-devs/pymc3/blob/master/pymc3/distributions/mixture.py
# https://docs.pymc.io/notebooks/model_comparison.html#Hierarchical-model
# https://github.com/pymc-devs/pymc3/tree/master/pymc3/examples
# http://people.duke.edu/~ccc14/sta-663-2016/16C_PyMC3.html

# with pm.Model() as weibull_2seg_2clust_withspike: #TODO: Has bugs fix it later
#     gamma_params = pm.Uniform('gamma_params', lower=0.0, upper=30.0, shape=(2,)) #alpha, beta
#     shape = pm.Uniform('shape', lower=0.0, upper=30.0)
#     scale_arr = pm.Gamma('scale_arr', alpha=gamma_params[0], beta=gamma_params[1], shape=(2,))
#     weibull = pm.Weibull('weibull', alpha=shape, beta=scale_arr, shape=(2,))
    
#     #obs = pm.Normal('obs', mu=10 * weibull[0] + 10 * weibull[1], sd=10, observed=ts_data[0][0:6]) 
#     #trace = pm.sample(1000) #pm.sample(niter, step=step, start=start, njobs=4, random_seed=123) 
    
#     # As we just need the logp, rather than add a RV to the model, we need to call .dist()
#     #components = pm.Poisson.dist(mu=lam, shape=(2,))  
#     w = pm.Dirichlet('w', a=np.array([1, 1]))
#     like = pm.Mixture('like', w=w, comp_dists=weibull, observed=ts_data[0])

import pymc3 as pm
import numpy as np

#Use iterable of distributions instead of array of random variables
with pm.Model() as weibull_2seg_2clust_withspike:
    gamma_alpha = pm.Uniform('gamma_alpha', lower=0.0, upper=30.0)
    gamma_beta = pm.Uniform('gamma_beta', lower=0.0, upper=30.0)
    shape1 = pm.HalfNormal('shape1', 1.)
    scale1 = pm.Gamma('scale1', alpha=gamma_alpha, beta=gamma_beta)
    shape2 = pm.HalfNormal('shape2', 1.)
    scale2 = pm.Gamma('scale2', alpha=gamma_alpha, beta=gamma_beta)   
    #weibull1 = pm.Weibull('weibull1', alpha=shape1, beta=scale1)
    #weibull2 = pm.Weibull('weibull2', alpha=shape2, beta=scale2)   
    #obs = pm.Normal('obs', mu=weibull1 + weibull2, sd=10, observed=[1,2,3,4,5])
    weibull1 = pm.Weibull.dist(alpha=shape1, beta=scale1)
    weibull2 = pm.Weibull.dist(alpha=shape2, beta=scale2)
    final_cluster_prob = pm.Uniform.dist(lower=0.0, upper=1.0)
    
    w1 = pm.Dirichlet('w1', a=np.array([1., 1., 1.]))
    obs1 = pm.Mixture('obs1', w=w1, comp_dists=[weibull1, weibull2, final_cluster_prob], observed=[1,2,3,4,5])
    w2 = pm.Dirichlet('w2', a=np.array([1., 1., 1.]))
    obs2 = pm.Mixture('obs2', w=w2, comp_dists=[weibull1, weibull2, final_cluster_prob], observed=[6,7,8,9,10])
    
    trace = pm.sample(1000)
    
    #inference = pm.fit(method='advi')  #This is mean field ADVI. Something like 'fullrank_advi' might be tried too 
    #trace = inference.sample(1000)
    
    #start = pm.find_MAP() # Find good starting point
    #step = pm.Slice() # Instantiate MCMC sampling algorithm
    #trace = pm.sample(1000, step, start=start, progressbar=True) # draw posterior samples using slice sampling     


In [ ]:
pm.summary(trace)

In [ ]:
# #Tinker with custom Weibull Gamma distribution in PyMC
# def logp(failure, value):
#     return (failure * log(λ) - λ * value).sum()

# exp_surv = pm.DensityDist('exp_surv', logp, observed={'failure':failure, 'value':t})

# Weibull fit to data
with pm.Model():
    data = [2,6,5,4,3] #[3,4,5,6,2]
    alpha = pm.HalfNormal('alpha')
    beta = pm.HalfNormal('beta')
    like = pm.Weibull('observation', alpha, beta, observed=data)
    start = pm.find_MAP()
    step = pm.Metropolis()
    trace = pm.sample(1000, step, start=start)

display(pm.summary(trace))
pm.traceplot(trace), plt.show()

In [ ]:
# with pm.Model() as model:
#     lam = pm.Exponential('lam', lam=1, shape=(2,))  # `shape=(2,)` indicates two mixtures.
#     # As we just need the logp, rather than add a RV to the model, we need to call .dist()
#     components = pm.Poisson.dist(mu=lam, shape=(2,))
#     w = pm.Dirichlet('w', a=np.array([1, 1]))  # two mixture component weights.
#     like = pm.Mixture('like', w=w, comp_dists=components, observed=[1,2,3,6,4,3,2])
#     trace = pm.sample(1000)

## Mixture example
# with pm.Model() as model:
#     lam = pm.Exponential('lam', lam=1, shape=(2,))  # `shape=(2,)` indicates two mixtures.
#     # As we just need the logp, rather than add a RV to the model, we need to call .dist()
#     components = pm.Poisson.dist(mu=lam, shape=(2,))
#     w = pm.Dirichlet('w', a=np.array([1, 1]))  # two mixture component weights.
#     like = pm.Mixture('like', w=w, comp_dists=components, observed=[1,2,3,6,4,3,2])
#     trace = pm.sample(1000)

## Compute MAP estimate
# map_estimate = pm.find_MAP(model=weibull_2seg_2clust_withspike, fmin=so.fmin_powell)
# print(map_estimate)

## Specifying start, sampler (step) and multiple chains
# with weibull_2seg_2clust_withspike:
#     start = pm.find_MAP()
#     step = pm.Metropolis()
#     start_step_trace = pm.sample(1000, step=step, start=start, njobs=4, random_seed=123)

## Merging Traces. TODO: This has some issue. Fix later
# from pymc3.backends.base import merge_traces
# with weibull_2seg_2clust_withspike:
#     merg_trace = merge_traces([pm.sample(1000, step, chain_idx=i) for i in range(4)])


In [ ]:
## Analyze sampling results

# db = pm.backends.Text('test')
# trace = pm.sample(..., trace=db)

tmp_trace = trace

print(tmp_trace.nchains)
pm.traceplot(tmp_trace), plt.show()
display(pm.summary(tmp_trace))
# display(pm.summary(tmp_trace[500:], varnames=['shape']))
pm.forestplot(tmp_trace), plt.show()
print(pm.effective_n(tmp_trace[500:]))
pm.plot_posterior(trace), plt.show()
# pm.autocorrplot(tmp_trace[500:], varnames=['shape']), plt.show()


## Tinker with Weibull and Weibull Gamma Distribution

In [ ]:
def weib(x, scale, shape):
    return (shape/scale) * (x/scale)**(shape-1) * np.exp(-(x/scale)**shape)

g_shape, g_scale = 5., 2. #3., 0.5152451259005756

data_x = np.arange(1,200.)/50.
data_prob = weib(data_x, g_scale, g_shape) #data, scale, shape
plt.plot(data_x, data_prob)

plt.figure()
raw_values = pm.Weibull.dist(alpha=g_shape, beta=g_scale).random(size=1000) #alpha:shape, beta:scale
# raw_values = 2*np.random.weibull(5.,1000) #specify shape, scale is fixed at 1
count, bins, ignored = plt.hist(raw_values, bins=10)
 
scale = count.max()/data_prob.max()
print(count.max(), data_prob.max(), scale)
plt.plot(data_x, data_prob*scale)


In [ ]:
# Custom weibull function fit using scipy optimizer

# https://github.com/scipy/scipy/blob/master/scipy/optimize/_minimize.py
# http://blog.mmast.net/optimization-scipy

from scipy.optimize import minimize, fmin

def weib(x, scale, shape):
    return (shape/scale) * (x/scale)**(shape-1) * np.exp(-(x/scale)**shape)

# def custom_weibull(x, lamb, c, max_len):
#     cdf = lambda x, lamb, c: 1.0 - np.exp(-lamb * np.float_power(x, c))
#     if x == 1:
#         return cdf(x, lamb, c)
#     if x == max_len:
#         return 1 - cdf(x-1, lamb, c)
#     return cdf(x, lamb, c) - cdf(x-1, lamb, c)

# def weibull_neg_loglike(params, tickets, half=False):
#     prob = lambda w, x, lamb, c, max_len: -w * np.log(custom_weibull(x, lamb, c, max_len))
#     # weibull requires x > 0; so, send i + 1 instead of i
#     return np.sum([prob(tickets[i], i + 1, params[0], params[1], len(tickets)) for i in range(len(tickets))])

bin_width = 50.
data_x = np.arange(1,200.)/bin_width
data_prob = weib(data_x, 2., 5.) #data, scale, shape

def weib_neg_loglike_ver1(theta):
    prob = lambda w, x, scale, shape: -w * np.log( (shape/scale) * (x/scale)**(shape-1) * np.exp(-(x/scale)**shape) )
    return np.sum([prob(data_prob[i], i+1, theta[0], theta[1]) for i in range(len(data_prob))])

# https://scholarsarchive.byu.edu/cgi/viewcontent.cgi?referer=https://www.google.com/&httpsredir=1&article=3508&context=etd
def weib_neg_loglike_ver2(theta): #Giving buggy results fix later
    n, scale, shape = len(data_prob), theta[0], theta[1]
    sum_1, sum_2 = 0., 0.
    for i in range(1,len(data_prob)+1):
        sum_1 = sum_1 + np.log(data_prob[i-1])
        sum_2 = sum_2 + (data_prob[i-1]/scale)**shape
    return -1* ( n*np.log(shape) - shape*n*np.log(scale) + (shape-1)*sum_1 - sum_2 )

theta_start = [100., 10.] #scale, shape
res = minimize(weib_neg_loglike_ver1, theta_start, method='Nelder-Mead', options={'disp':True})
# res = fmin(weib_neg_loglike_ver1, theta_start, xtol = 0.0000001, ftol = 0.0000001, disp = 1)
print(res)
print('Shape:{}|Scale:{}|RShape:{}|RScale:{}'.format(res.x[1],(res.x[0]/bin_width),round(res.x[1]),round(res.x[0]/bin_width))) 


In [ ]:
#Weibull Gamma optimization on simulated data

# F(t|lambda,c) = 1 - e^(-lambda*t^c)
# g(lambda|r,alpha) = alpha^r*lambda^(r-1)*e^(-alpha*lambda) / Gamma(r)
# https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.random.gamma.html

# def weib(x, scale, shape):
#     return (shape/scale) * (x/scale)**(shape-1) * np.exp(-(x/scale)**shape)

simulated_data = np.array([])
w_shape, g_r, g_alpha =  3., 10., 1./0.9 #w-shape, g-shape, g-rate
lamba_arr = np.random.gamma(shape=g_r, scale=1.0/g_alpha, size=1)
for lambda_v in lamba_arr:
    w_scale = np.exp(-np.log(lambda_v)/w_shape)
    bin_width = 50.
    data_x = np.arange(1,200.)/bin_width
    data_prob = weib(data_x, w_scale, w_shape) #data, scale, shape
    #print(lambda_v, w_scale, count.max(), data_prob.max(), tmp_scale)
    plt.plot(data_x, data_prob)
    #simulated_data = np.append(simulated_data, pm.Weibull.dist(alpha=w_shape, beta=w_scale).random(size=1000))
    simulated_data = data_prob

def weib_gamma_neg_loglike(theta):
    def prob(w, x, w_shape, g_shape, g_scale):
        w_scale_var = ss.gamma(g_shape, scale=g_scale)
        w_scale = w_scale_var.mean()
        res = -w * np.log( (w_shape/w_scale) * (x/w_scale)**(w_shape-1) * np.exp(-(x/w_scale)**w_shape) )
        return res
    return np.sum([prob(simulated_data[i], i+1, theta[0], theta[1], theta[2]) for i in range(len(simulated_data))])

theta_start = [4., 11., 12.5] #w-shape, g-shape, g-scale
res = minimize(weib_gamma_neg_loglike, theta_start, method='Nelder-Mead', options={'disp':True}) #method='Nelder-Mead', 
print(res)

#true param : [3., 10., 0.9] #w-shape, g-shape, g-rate
#start:[4., 11., 12.5], end:[2.07498354, 10.5404605 , 12.09560527], method:default (bfgs ...)
#start:[4., 11., 12.5], end:[2.95151024, 4.70356652, 4.50075516], method:Nelder-Mead
#start:[5., 12., 5.], end:[4.33672378, 11.70704407,  4.29690579], method:default (bfgs ...)
#start:[5., 12., 5.], end:[3.00004045, 15.72523612,  1.46519853], method:Nelder-Mead


In [ ]:
#Optimize mixture of weibull cdf

def weibull_2seg_2clust_withspike_negloglike(theta):
    prob = lambda x, y, z: -x * np.log(gamma.pdf(y, z))
    data = [10, 9, 8, 7, 6, 5]
    return np.sum([prob(data[i], i+1, theta) for i in range(len(data))])


params = fit_weibull_2seg_2clust_withspike(params, [clust1_ts, clust2_ts])

In [ ]:
##scipy stats exponweib distribution fit to dummy data

data = [10,9,8,7,6,5,4,4,4,5,6,7,7,7]
plt.plot(data, ss.exponweib.pdf(data, *ss.exponweib.fit(data, 1, 1, scale=2, loc=0)))
_ = plt.hist(data, bins=np.linspace(0, 16, 33), normed=True, alpha=0.5)
plt.show()

In [ ]:
##PyMC distribution

# y = pm.Weibull.dist(alpha=20, beta=1000) #shape, scale
# print(y.logp(1000).eval())
# sns.distplot(y.random(size=1000))

# y1 = pm.Weibull.dist(alpha=0.9, beta=10) 
# y2 = pm.Weibull.dist(alpha=20, beta=1000)
# # y3 = pm.Weibull.dist(alpha=1, beta=500)
# raw_values = np.append(y1.random(size=1000), y2.random(size=1000))
# sns.distplot(raw_values)
# plt.figure(), plt.hist(raw_values)


y1 = pm.Weibull.dist(alpha=2, beta=10) 
raw_values = y1.random(size=1000)
sns.distplot(raw_values)
plt.figure(), plt.hist(raw_values)


## Plot Gamma and Gamma Gamma Distribution

In [ ]:
# http://www.brucehardie.com/notes/025/gamma_gamma.pdf

import scipy.special as sps
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt

# shape, scale = 1.55, 3.9  # mean=4, std=2*sqrt(2)
# s = np.random.gamma(shape, scale, 1000)
# count, bins, ignored = plt.hist(s, 50, normed=True)
# y = bins**(shape-1)*(np.exp(-bins/scale) / (sps.gamma(shape)*scale**shape))
# plt.plot(bins, y, linewidth=2, color='r')
# plt.show()

## Plot Gamma-Gamma function
p, q, g = 2.8, 14.8, 441.7
nu = np.random.gamma(q, 1.0/g, 10)
for nu_val in nu:
    s = np.random.gamma(p, 1.0/nu_val, 1000)
    shape, scale = p, 1.0/nu_val 
    count, bins, ignored = plt.hist(s, 50, normed=True)
    y = bins**(shape-1)*(np.exp(-bins/scale) / (sps.gamma(shape)*scale**shape))
    plt.plot(bins, y, linewidth=2, color='r')

## Gamma Gamma Regression (tinker with tensorflow or  pytorch or scipy distr or statsmodel)

In [ ]:
tmp_list_list = [['a','b',5,6,7,8,7,6,5,4,3,2,1,1],['c','d',6,6,7,7,8,9,6,3,3,3,1,1],['a','b',7,8,9,10,9,8,7,6,5,4,3,3],['c','d',9,9,10,10,11,12,9,6,6,6,4,4]] 
tmp_df = pd.DataFrame(tmp_list_list) 
tmp_df.columns = list(map(lambda x : 'c'+str(x), tmp_df.columns))

freq = np.array(tmp_df.iloc[0][2:].values.tolist())
val = np.array(list(map(lambda x : int(x[1:]), tmp_df.columns[2:].tolist())))

tmp_df

In [ ]:
from lifetimes import GammaGammaFitter

ggf = GammaGammaFitter(penalizer_coef = 0)
ggf.fit(freq, val)
print(ggf)


## Poisson, Exponential Regression (tensorflow, pytorch, scipy distr, statsmodel)

## Tinker with Beta Binomial Distribution (PyMC)

In [ ]:
tmp_df = pd.DataFrame([['a',10,5],['b',10,2],['c',12,2],['d',20,5],['e',100,1]], columns=['scenario','sample_size','conversions'])
tmp_df['cr'] = 1.0 * tmp_df['conversions'] / tmp_df['sample_size']
num_samples = len(tmp_df)

print(tmp_df.shape)
display(tmp_df['conversions'].describe())
alpha1, beta1, loc1, scale1 = ss.beta.fit(tmp_df['cr'][0:500])
print(alpha1, beta1, loc1, scale1)

In [ ]:
# https://docs.pymc.io/notebooks/hierarchical_partial_pooling.html
# https://stackoverflow.com/questions/43710346/difference-between-betabinomial-and-beta-and-binomial

sample_size_arr, obs_arr = tmp_df['sample_size'].values, tmp_df['conversions'].values

with pm.Model() as bb:
    scenarios = pm.Beta('scenarios', alpha=alpha1, beta=beta1, shape=num_samples)
    cr_like = pm.Binomial('cr_like', n=sample_size_arr, p=scenarios, observed=obs_arr)

    #alpha0 = pm.Uniform('alpha', 1, 100)
    #beta0 = pm.Uniform('beta', 1, 100)
    #cr_like = pm.BetaBinomial('cr_like', alpha=alpha0, beta=beta0, n=sample_size_arr, observed=obs_arr)

with bb:
    hierarchical_trace = pm.sample(draws=1000)#, n_init=1000)

tmp_trace = hierarchical_trace
summary_df = pm.summary(tmp_trace[500:])
summary_df['mean'].describe(), tmp_df['cr'].describe()

In [ ]:
summary_df